In [7]:
import collections
import json
import os
import math
import pandas
from pandas import DataFrame

In [8]:

def json_file_list(path: str, sections: list) -> dict:
    """
    Lista o conteúdo de uma determinada pasta e retorna um dicionário com o conteúdo referente a cada pasta de cada seção.
    """
    file_list_dict = dict.fromkeys(sections)
    for section in sections:
        file_list_dict[section] = os.listdir(path + section)
    return file_list_dict


def read_json(filename: str) -> dict:
    """
    Carrega o arquivo JSON.
    """
    try:
        with open(filename, "r") as f:
            data = json.loads(f.read())
    except:
        raise Exception(f"Reading {filename}: file encountered an error")
    return data


def read_file(path: str, section: str) -> list:
    """
    Carrega o arquivo TXT.
    """
    if section in path:
        try:
            with open(path, "r") as f:
                data = f.readlines()
        except:
            raise Exception(f"Reading {path} file encountered an error")
        return data


def save_dataframe(dataframe: DataFrame, path: str, section: str):
    """
    Salva o dataframe no formato CSV.
    """
    dataframe.to_csv(path + section + "mnr.csv", index=False)  # Gravei o norevil como revil, tem que trocar o nome e rodar denovo pra gravar o revil certo


def flatten(d: dict, sep=".") -> collections.OrderedDict:
    """
    Transforma o json aninhado num dicionário com os nós da árvore concatenados em um único nível, com ponto como separador das chaves do dicionário original
    """
    # import collections
    # obj = {}
    obj = collections.OrderedDict()
    def recurse(t, parent_key=""):
        if isinstance(t, list):
            for i in range(len(t)):
                recurse(t[i], parent_key + sep + str(i) if parent_key else str(i))
        elif isinstance(t, dict):
            for k, v in t.items():                
                recurse(v, parent_key + sep + k if parent_key else k)
        else:
            obj[parent_key] = t
    recurse(d)
    return obj


def normalize(flat_json):
    """
    Função que transforma o flat json em dataframe.
    """
    keys_list = list(flat_json.keys())
    unique_cols = {}
    for items in keys_list:
        api = items.split(sep='.')[-1]
        if api not in unique_cols:
            unique_cols[api] = 0
    for k, v in flat_json.items():
        k_split = k.split(sep='.')[-1]
        if k_split in unique_cols:
            unique_cols[k_split] = unique_cols[k_split] + v
    df1 = pandas.json_normalize(unique_cols)
    return df1


def compute_tf(word_dict):
    """
    Calcula a frequência de termos de um documento. A frequência de termos é a quantidade de vezes que aquele termo aparece sobre a quantidade de termos que tem naquele documento.
    """
    tf_dict = {}
    word_count = len(word_dict.keys())
    for word, count in word_dict.items():
        tf_dict[word] = count / float(word_count)
    return tf_dict


def compute_idf(corpus: dict, wordlist: list) -> dict:
    """
    Calcula o inverso da quantidade de documentos em que determinado termo aparece, utilizando a fórmula IDF(t) = log_e(Total number of documents / Number of documents with term t in it).
    """
    idf_dict = dict.fromkeys(wordlist, 0.0)
    N = len(corpus.keys())
    for documents in corpus:
        for word, value in corpus[documents].items():
            if value > 0:
                idf_dict[word] += 1
    for word, value in idf_dict.items():
        idf_dict[word] = math.log10(((1 + N) / (1 + value)) + 1)
    return idf_dict


def compute_tfidf(tf: dict, idf: dict) -> dict:
    """
    Calcula o TF-IDF do corpus (TF*IDF de cada termo).
    """
    tf_idf = {}
    for document_id in tf:
        tf_idf[document_id] = {}
        for term, value in tf[document_id].items():
            tf_idf[document_id][term] = value * idf[term]
    return tf_idf


def create_wordlist(corpus: dict) -> list:
    """
    Cria a lista de termos a partir do corpus de documentos (dicionário).
    """
    wordlist = []
    
    """
    # Este trecho, apesar de ter a lógica bastante óbvia, não estava sendo eficiente para a geração da wordlist.
    for documents in corpus:
        for term in corpus[documents].keys():
            if term not in wordlist:
                wordlist.append(term)
    """
    # Confecciona a wordlist de maneira mais eficiente que iterar cada termo em cada documento e comparar um a um com toda a wordlist.
    temp_list = []
    for documents in corpus:
        for term in corpus[documents].keys():
            temp_list.append(term)
    wordlist = list(dict.fromkeys(temp_list))
    return wordlist


def convert_to_dataframe(tf_idf: dict):
    """
    Converte o dicionário com o tf_idf em um dataframe.
    """
    df1 = pandas.DataFrame
    cont = 0
    sample_id = sorted(list(tf_idf.keys()))
    for report in sample_id:
        if cont == 0:
            print(cont, "- Preparando o dicionário da amostra", report, "para transformar em Dataframe", end='')
            flatted = flatten(tf_idf[report])
            df1 = normalize(flatted)
            flatted = None
            print("......finalizado")
        else:
            print(cont, "- Preparando o dicionário da amostra", report, "para transformar em Dataframe", end='')
            flatted = flatten(tf_idf[report])
            df2 = normalize(flatted)
            flatted = None
            print("......finalizado")

            print("Concatenando Dataframes", end='')
            table = pandas.concat([df1, df2], ignore_index=True, axis=0)
            df1 = table
            print("......finalizado")
            
            print("DataFrame com", len(table.columns), "colunas")
            
            print("Limpando memória", end='')
            table = None
            df2 = None
            print("......finalizado")
        cont += 1
    df1["id"] = sample_id
    return df1


def df_process(table: DataFrame) -> DataFrame:
    """
    FSubstitui os valores N/A por zero no dataframe.
    """
    for column in table.columns:
        table[column].fillna(0, inplace=True)
    return table


def strip_chars(dictionary: dict) -> dict:
    """
    Função coringa, usada para implementar os processamentos intermediários no dataframe.
    
    Refaz as strings que são as chaves do dicionário sem os pontos para não interferir na função flatten.
    
    """
    
    #stripped_dict = {}
    #char_to_replace = {'.', ',', ' ', '\n', '\r', '?', '_', '!', '-', '*'}    # ponto virgula e espaço por nada
    #for char in char_to_replace:
    #    stripped_dict = {k.replace(char, ''): v for k, v in dictionary.items()}
    
    # tentativa de substituir as strings por hashes
    hashed_dict = {}
    for key, value in dictionary.items():
        hashed_dict[str(hash(key))] = value
    
    return hashed_dict
    # return stripped_dict


In [9]:
sections = ['behavior']  

path = "/content/drive/MyDrive/Corpus/"
    
tf, idf, tf_idf = {}, {}, {}

print("Obtendo lista de arquivos na pasta de seção", end='')
file_list_dict = json_file_list(path, sections)  
print("............completo")

for section in sections:
  
  wordlist = []
  
  section_corpus = {}
  
  for file in file_list_dict[section]:    
      document_id = int(file[0:4])  

      if (452 <= document_id <=  468) or (480 <= document_id <= 611) or (712 <= document_id <= 784): # Seleção das partes do corpus para compor o dataframe

      # NoREVIL: (document_id <= 611) or (711 <= document_id <= 784) # Depois que adicionou o goodware, o arquivo ficou com 700MB e o colab nao consegue carregar

      # REVIL: (612 <= document_id <=  784) or (791 <= document_id <= 1446)

      # CLOP CONI EGREGOR LOCKBIT: (document_id <=  451) or (712 <= document_id <= 784)

      # MOUNTLOCKER NETWALKER RYUK: (452 <= document_id <=  468) or (480 <= document_id <= 611) or (712 <= document_id <=  784)

        file_path = path + section + "//" + file  
        
        print("Carregando arquivo", file, end="")
        document_json = read_json(file_path)
        stripped_document = strip_chars(document_json)
        print("..............concluído")

        print("Calculando frequencia de termos (TF)", end="")
        tf[document_id] = compute_tf(stripped_document)
        print("..............concluído")

        print("Criando o section_corpus com os documentos", end="")
        section_corpus[document_id] = stripped_document
        print("..............concluído")

  print("Criando wordlist", end="")
  wordlist = create_wordlist(section_corpus)
  print("..............concluído")
  print("Wordlist com " + str(len(wordlist)) + " termos")
  
  print("Calculando a frequência inversa dos termos nos documentos (IDF)", end="")
  idf = compute_idf(section_corpus, wordlist)
  print("..............concluído")

  print("Calculando TF-IDF", end="")
  tf_idf = compute_tfidf(tf, idf)
  print("..............concluído")
  
  print("Transformando a tabela", section, "em dataframe")
  dataframe_tf_idf = convert_to_dataframe(tf_idf)
  
  print("Ajustando o dataframe", end="")
  processed_dataframe_tf_idf = df_process(dataframe_tf_idf)
  print("..............concluído")
  
  print("Gravando arquivo do dataframe em disco", end="")
  save_dataframe(processed_dataframe_tf_idf, path, section)
  print("..............concluído\n")        

Obtendo lista de arquivos na pasta de seção............completo
Carregando arquivo 454 - behavior.json..............concluído
Calculando frequencia de termos (TF)..............concluído
Criando o section_corpus com os documentos..............concluído
Carregando arquivo 452 - behavior.json..............concluído
Calculando frequencia de termos (TF)..............concluído
Criando o section_corpus com os documentos..............concluído
Carregando arquivo 453 - behavior.json..............concluído
Calculando frequencia de termos (TF)..............concluído
Criando o section_corpus com os documentos..............concluído
Carregando arquivo 456 - behavior.json..............concluído
Calculando frequencia de termos (TF)..............concluído
Criando o section_corpus com os documentos..............concluído
Carregando arquivo 455 - behavior.json..............concluído
Calculando frequencia de termos (TF)..............concluído
Criando o section_corpus com os documentos..............conclu

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:174: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


Ajustando o dataframe..............concluído
Gravando arquivo do dataframe em disco..............concluído



In [10]:
# Verifica se a tabela foi confeccionada corretamente
import pandas
file_path = "/content/drive/MyDrive/Corpus/behaviornorevil.csv"
with open(file_path,"r") as f:
    table = pandas.read_csv(f)
table

FileNotFoundError: ignored

In [ ]:
family = []

for i in range(len(table)):
    if 192<=table["id"][i]<=206:
        family.append("clop")
    elif(207<=table["id"][i]<=310):
        family.append("conti")
    elif(311<=table["id"][i]<=355):
        family.append("egregor")
    elif(403<=table["id"][i]<=435):
        family.append("lockbit")
    elif(438<=table["id"][i]<=451):
        family.append("lockbit")
    elif(452<=table["id"][i]<=468):
        family.append("mountlocker")
    elif(480<=table["id"][i]<=557):
        family.append("netwalker")
    elif(559<=table["id"][i]<=611):
        family.append("ryuk")
    elif(612<=table["id"][i]<=711):
        family.append("revil")
    elif(712<=table["id"][i]<=784):
        family.append("goodware")
    elif(791<table["id"][i]<1446):
        family.append("revil")
    else:
        family.append("unknown")

In [ ]:
for i in range(len(family)):
    print(table['id'][i], family[i])